<a href="https://colab.research.google.com/github/Forbusinessuseyukikoishiguro/20250327_chatgpt_peech_summarizer/blob/main/%E2%91%A0AI%E3%82%A8%E3%83%B3%E3%82%B8%E3%83%8B%E3%82%A2%E8%BB%A2%E8%81%B7%E7%94%A8%E3%83%9D%E3%83%BC%E3%83%88%E3%83%95%E3%82%A9%E3%83%AA%E3%82%AA_%E3%81%8A%E5%AC%A2%E6%A7%98%E3%81%AE%E5%9F%B7%E4%BA%8B%E3%83%81%E3%83%A3%E3%83%83%E3%83%88%E3%83%9C%E3%83%83%E3%83%8820250402.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ｓｓ「

In [ ]:
# 必要なライブラリをインストール
!pip install openai python-dotenv

# ライブラリをインポート
import os
import re
import random
from openai import OpenAI
from google.colab import userdata  # Google Colabのユーザーデータ機能
from IPython.display import clear_output

# 執事の応答パターン
butler_responses = {
    "positive": "お嬢様、さすがです。あなたの努力が実を結びましたね。今後もその調子で、どんどん素晴らしい成果を上げていかれることでしょう。",
    "tired": "お嬢様、少しお休みになられては？無理をしすぎることはありません。あなたのご健康が一番大切ですから、少しだけリラックスなさってください。",
    "angry": "お嬢様、今は少し落ち着いて深呼吸をしてみてください。あまりご自分を責めすぎないように。どんなに辛いことでも、時間が解決してくれることが多いものです。",
    "hard": "お嬢様、今はとても辛いお気持ちかと思いますが、必ずその先に明るい日が待っています。私も常にお側におりますから、少しでもお力になれればと思います。",
    "sad": "お嬢様、あなたの気持ちが痛いほど分かります。涙を流すことは決して弱さではありません。その気持ちを大切にし、ゆっくりと癒していきましょう。私もここでお力になります。",
    "progress": "お嬢様、これはほんの一歩に過ぎません。あなたの道のりはまだ続いていますが、どんな困難にも立ち向かう力があることを、私はよく知っています。これからも自信を持って進んでいってください。",
    "other": "お嬢様、いつも素晴らしいお姿に感銘を受けております。どのような時も、私はあなたの執事として心を込めてお仕えいたします。"
}

# 感情キーワード辞書（バックアップ用）
emotion_keywords = {
    "tired": ["疲れ", "つかれ", "くたびれ", "眠い", "ねむい", "しんどい", "きつい"],
    "angry": ["腹が立つ", "怒り", "むかつく", "イライラ", "頭に来る", "キレる", "うざい"],
    "hard": ["辛い", "つらい", "苦しい", "くるしい", "大変", "きびしい", "厳しい"],
    "sad": ["悲しい", "かなしい", "寂しい", "さみしい", "切ない", "せつない", "泣きたい"],
    "progress": ["うまくいく", "順調", "成功", "達成", "できた", "やりました", "前進"],
    "positive": ["嬉しい", "うれしい", "幸せ", "しあわせ", "楽しい", "たのしい", "ハッピー"]
}

# 会話履歴を保存
conversation_history = []

# API key setup
try:
    # Get API key from Google Colab's secure storage
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    if OPENAI_API_KEY:
        # Initialize the client with the API key
        client = OpenAI(api_key=OPENAI_API_KEY)
        print("API key loaded successfully.")
        api_available = True
    else:
        raise ValueError("API key not set")
except Exception as e:
    print(f"Failed to load API key: {e}")
    print("Will use keyword-based responses instead.")
    api_available = False

# Function to detect if input is emotional or a general question
def is_emotional_input(input_text):
    for emotion, keywords in emotion_keywords.items():
        for keyword in keywords:
            if keyword in input_text:
                return True
    return False

# Function to generate response using GPT model for general questions
def get_gpt_general_response(input_text):
    try:
        # Add to conversation history
        conversation_history.append({"role": "user", "content": input_text})

        # Prepare conversation for API
        messages = [
            {"role": "system", "content": "You are an elegant and polite butler speaking to your mistress. Always respond in Japanese with a polite, sophisticated tone. Address the user as 'お嬢様' and be helpful, knowledgeable, and supportive. Your responses should reflect a butler's dedication and respect."}
        ]

        # Add conversation history (up to last 10 exchanges)
        messages.extend(conversation_history[-10:])

        # Using the new API format
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=200,
            temperature=0.7
        )

        butler_reply = response.choices[0].message.content.strip()

        # Add to conversation history
        conversation_history.append({"role": "assistant", "content": butler_reply})

        return butler_reply

    except Exception as e:
        print(f"GPT API error: {e}")
        return "お嬢様、申し訳ございません。ただいま一時的にお答えできない状況でございます。少々お時間をいただければ幸いです。"

# Function to generate response for emotional inputs using GPT
def get_gpt_emotional_response(input_text):
    try:
        prompt = f"""
You are roleplaying as a butler. Please respond to the following statement from your mistress with an appropriate response.
Choose a response based on the mistress's emotion.

1. When feeling successful:
「お嬢様、さすがです。あなたの努力が実を結びましたね。今後もその調子で、どんどん素晴らしい成果を上げていかれることでしょう。」

2. When feeling tired:
「お嬢様、少しお休みになられては？無理をしすぎることはありません。あなたのご健康が一番大切ですから、少しだけリラックスなさってください。」

3. When feeling angry:
「お嬢様、今は少し落ち着いて深呼吸をしてみてください。あまりご自分を責めすぎないように。どんなに辛いことでも、時間が解決してくれることが多いものです。」

4. When feeling distressed:
「お嬢様、今はとても辛いお気持ちかと思いますが、必ずその先に明るい日が待っています。私も常にお側におりますから、少しでもお力になれればと思います。」

5. When feeling sad:
「お嬢様、あなたの気持ちが痛いほど分かります。涙を流すことは決して弱さではありません。その気持ちを大切にし、ゆっくりと癒していきましょう。私もここでお力になります。」

6. When feeling progress:
「お嬢様、これはほんの一歩に過ぎません。あなたの道のりはまだ続いていますが、どんな困難にも立ち向かう力があることを、私はよく知っています。これからも自信を持って進んでいってください。」

7. Other:
Choose words that fit the role of a butler supporting your mistress with a warm heart.

Mistress's statement: 「{input_text}」

From the above statement, guess the mistress's emotion and return only the most appropriate response.
Format: Return exactly one of the response texts above. Do not create new text.
"""

        # Using the new API format
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an elegant and polite butler."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=200,
            temperature=0.3
        )

        # Extract appropriate part from GPT's response
        butler_reply = response.choices[0].message.content.strip()

        # Match with predefined responses
        for response_text in butler_responses.values():
            if response_text in butler_reply:
                return response_text

        # If no predefined response is found, return the GPT response (only parts starting with "お嬢様")
        if "お嬢様" in butler_reply:
            match = re.search(r'(お嬢様.*)', butler_reply)
            if match:
                return match.group(1)

        # If still no appropriate response, return default
        return butler_responses["other"]

    except Exception as e:
        print(f"GPT API error: {e}")
        print("Switching to keyword-based response.")
        return get_keyword_response(input_text)

# Function to detect emotion and return response based on keywords (backup)
def get_keyword_response(input_text):
    # Keyword-based emotion detection
    for emotion, keywords in emotion_keywords.items():
        for keyword in keywords:
            if keyword in input_text:
                return butler_responses[emotion]

    # Default response if no keyword is found
    return butler_responses["other"]

# Function to generate butler response
def get_butler_response(input_text):
    # Check if input is emotional or general question
    if is_emotional_input(input_text) and api_available:
        # Emotional input and API available
        return get_gpt_emotional_response(input_text)
    elif is_emotional_input(input_text) and not api_available:
        # Emotional input but API not available
        return get_keyword_response(input_text)
    elif api_available:
        # General question and API available
        return get_gpt_general_response(input_text)
    else:
        # General question but API not available
        return "お嬢様、申し訳ございませんが、その質問にはお答えする準備ができておりません。お力になれず心苦しく思います。"

# Simple interface for Google Colab
def chat_with_butler():
    print("===== お嬢様の執事チャットボット =====")
    print("※会話を終了するには「終了」と入力してください")

    if not api_available:
        print("\nNote: API key is not set, using basic emotion analysis only.")
        print("To use advanced emotion analysis, set the API key in Google Colab's 'Secrets' tab.\n")

    while True:
        user_input = input("\nお嬢様: ")

        if user_input == "終了":
            print("\n執事：またのお呼び出しをお待ちしております、お嬢様。良き一日をお過ごしください。")
            break

        response = get_butler_response(user_input)
        print(f"\n執事: {response}")

# Start the chatbot
chat_with_butler()

API key loaded successfully.
===== お嬢様の執事チャットボット =====
※会話を終了するには「終了」と入力してください

執事: お嬢様、少しお休みになられては？無理をしすぎることはありません。あなたのご健康が一番大切ですから、少しだけリラックスなさってください。

執事: お嬢様、池袋に行かれるのですね。美味しいケーキをお楽しみにされるのは素晴らしいお選びです。お勧めのケーキ屋さんといたしましては、池袋エリアには「ル・タン・タン」や「パティスリー タカギ」などの人気店がございます。どちらもお嬢様にご満足いただけることでしょう。お時間がございましたら、ぜひお立ち寄りいただき、お楽しみくださいませ。何かお手伝いがございましたら、いつでもお申し付けください

お嬢様: チョコケーキが有名なとこはある？

執事: お嬢様、チョコレートケーキがお好きなのですね。チョコレートケーキで有名なお店としては、池袋エリアにも「シャンブル ドゥ セザンヌ」や「アンリ・シャルパンティエ」などがございます。どちらも素晴らしいチョコレートケーキをご提供しておりますので、ぜひお試しいただければと存じます。お嬢様のお好みに合ったお店を見つけられますよう、心よりお手伝いいたします。何かご質問がございましたら、お気軽にお
